# Global Daily Event Analysis: Marine Heatwave ID & Tracking using `MarEx`

In [1]:
import xarray as xr
import dask
from getpass import getuser
from pathlib import Path

import marEx
import marEx.helper as hpc

In [2]:
scratch_dir = Path('/scratch') / getuser()[0] / getuser()

In [3]:
# Start Dask Cluster
client = hpc.start_local_cluster(n_workers=4, threads_per_worker=1, memory_limit='2GB')  # Specify temporary scratch directory for dask to use

Dask Scratch: '/scratch/b/b382615/clients/tmp3aaw6t2h'
Memory per Worker: 125.92 GB
Hostname: l40125
Forward Port: l40125:8787
Dashboard Link: localhost:8787/status


In [4]:
test_data_path = "/home/b/b382615/opt/marEx/tests/data/extremes_gridded.zarr"
extremes_data = xr.open_zarr(str(test_data_path), chunks={}).persist()

# file_name = scratch_dir / 'mhws' / 'extremes_binary_gridded_shifting.zarr'
# extremes_data = xr.open_zarr(str(file_name), chunks=chunk_size)
# extremes_data = extremes_data.isel(time=slice(-32,-1))

In [5]:
extremes_data

<xarray.Dataset> Size: 2MB
Dimensions:         (time: 32, lat: 180, lon: 360)
Coordinates:
  * lat             (lat) float64 1kB -89.62 -88.62 -87.62 ... 87.38 88.38 89.38
  * lon             (lon) float64 3kB 0.375 1.375 2.375 ... 357.4 358.4 359.4
  * time            (time) datetime64[ns] 256B 2010-01-01T23:59:59 ... 2010-0...
Data variables:
    extreme_events  (time, lat, lon) bool 2MB dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    mask            (lat, lon) bool 65kB dask.array<chunksize=(180, 360), meta=np.ndarray>
Attributes:
    method_anomaly:        shifting_baseline
    method_extreme:        hobday_extreme
    preprocessing_steps:   ['Rolling climatology using 15 years', 'Smoothed w...
    smooth_days_baseline:  21
    threshold_percentile:  95
    window_days_hobday:    11
    window_year_baseline:  15

In [6]:
tracker = marEx.tracker(
            extremes_data.extreme_events,
            extremes_data.mask.where(
                (extremes_data.lat < 85) & (extremes_data.lat > -90), 
                other=False
            ),  # Exclude poles
            area_filter_quartile=0.5,
            R_fill=4,  # Reduced for test data
            T_fill=2,  # No temporal filling for basic test
            allow_merging=True,
            verbosity=1  # Suppress output for tests
        )

tracked_ds = tracker.run()

Finished filling spatial holes
Finished filling spatio-temporal holes
Finished filtering small objects
Finished object identification
Finished calculating object properties
Finished finding overlapping objects
Processing splitting and merging in chunk 0 of 16
Processing splitting and merging in chunk 10 of 16
Finished splitting and merging objects
Finished clustering and renaming objects into coherent consistent events
Finished tracking all extreme events!


Tracking Statistics:
   Binary Hobday to Processed Area Fraction: 0.70147268057429
   Total Object Area IDed (cells): 250371.0
   Number of Initial Pre-Filtered Objects: 461
   Number of Final Filtered Objects: 230
   Area Cutoff Threshold (cells): 358
   Accepted Area Fraction: 0.8708236976327131
   Total Events Tracked: 20
   Total Merging Events Recorded: 32


In [7]:
tracked_ds

<xarray.Dataset> Size: 8MB
Dimensions:       (lat: 180, lon: 360, time: 32, ID: 20, component: 2,
                   sibling_ID: 2)
Coordinates:
  * lat           (lat) float64 1kB -89.62 -88.62 -87.62 ... 87.38 88.38 89.38
  * lon           (lon) float64 3kB 0.375 1.375 2.375 ... 357.4 358.4 359.4
  * time          (time) datetime64[ns] 256B 2010-01-01T23:59:59 ... 2010-02-...
  * ID            (ID) int32 80B 1 2 3 4 5 6 7 8 9 ... 13 14 15 16 17 18 19 20
Dimensions without coordinates: component, sibling_ID
Data variables:
    ID_field      (time, lat, lon) int32 8MB dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    global_ID     (time, ID) int32 3kB dask.array<chunksize=(2, 20), meta=np.ndarray>
    area          (time, ID) float32 3kB dask.array<chunksize=(2, 20), meta=np.ndarray>
    centroid      (component, time, ID) float64 10kB dask.array<chunksize=(2, 2, 20), meta=np.ndarray>
    presence      (time, ID) bool 640B dask.array<chunksize=(2, 20), meta=np.ndarray>
    time_start    (ID) datetime64[ns] 160B dask.array<chunksize=(20,), meta=np.ndarray>
    time_end      (ID) datetime64[ns] 160B dask.array<chunksize=(20,), meta=np.ndarray>
    merge_ledger  (time, ID, sibling_ID) int32 5kB dask.array<chunksize=(2, 20, 2), meta=np.ndarray>
Attributes: (12/14)
    allow_merging:               1
    N_objects_prefiltered:       461
    N_objects_filtered:          230
    N_events_final:              20
    R_fill:                      4
    T_fill:                      2
    ...                          ...
    accepted_area_fraction:      0.8708236976327131
    preprocessed_area_fraction:  0.70147268057429
    overlap_threshold:           0.5
    nn_partitioning:             0
    total_merges:                32
    multi_parent_merges:         0

In [8]:
tracked_ds = tracked_ds.persist()

In [15]:
present_events = tracked_ds.presence.values

In [11]:
present_events.any(dim='time').values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [17]:
areas_np = tracked_ds.area.values
areas_at_present = areas_np[present_events]

In [19]:
assert (areas_at_present > 0).all()

In [27]:
present_events = tracked_ds.presence
present_events_np = present_events.values

In [21]:
lat_min, lat_max = float(extremes_data.lat.min()), float(extremes_data.lat.max())
lon_min, lon_max = float(extremes_data.lon.min()), float(extremes_data.lon.max())

In [26]:
lat_centroids = tracked_ds.centroid.sel(component=0).values  # Assuming lat is first component
lon_centroids = tracked_ds.centroid.sel(component=1).values

In [28]:
present_lat_centroids = lat_centroids[present_events_np]
present_lat_centroids

array([-55.09244156,  27.73458099,  19.30427551,  48.87553406,
       -55.32437515,  41.60997009,  15.5273056 ,  47.10116577,
       -54.55033112, -43.77758026, -55.02626038,  39.84729004,
        14.97662354,  47.36972046, -57.47855759, -42.70104599,
       -54.7567482 ,  38.60134888,  14.95629883,  49.79768372,
       -58.2359848 , -42.79931259, -15.06536865, -54.17849731,
        38.17084503,  15.30697632,  50.37763977, -58.11938286,
       -43.70649719, -11.03516388, -54.06079865,  36.42797089,
        15.48731232,  50.44384766, -58.42621613, -45.7984848 ,
         5.97018433, -53.69202805,  36.23554993,  16.74915314,
        50.55737305, -58.35255432, -45.75421524,   6.07695007,
       -12.05615997, -53.71590805,  37.8598175 ,  20.60158539,
        49.38508606, -58.52809334,   5.41500092, -15.80286407,
       -54.89497757,  39.82655334,  20.60166931,  49.59532166,
       -58.92876434,   5.54166412, -16.02197266, -55.44865417,
        39.80172729,  19.97240448,  49.64668274, -58.79

In [ ]:
present_lon_centroids = lon_centroids.where(present_events)

In [29]:
present_lat_centroids = lat_centroids[present_events.values]
present_lat_centroids

array([-55.09244156,  27.73458099,  19.30427551,  48.87553406,
       -55.32437515,  41.60997009,  15.5273056 ,  47.10116577,
       -54.55033112, -43.77758026, -55.02626038,  39.84729004,
        14.97662354,  47.36972046, -57.47855759, -42.70104599,
       -54.7567482 ,  38.60134888,  14.95629883,  49.79768372,
       -58.2359848 , -42.79931259, -15.06536865, -54.17849731,
        38.17084503,  15.30697632,  50.37763977, -58.11938286,
       -43.70649719, -11.03516388, -54.06079865,  36.42797089,
        15.48731232,  50.44384766, -58.42621613, -45.7984848 ,
         5.97018433, -53.69202805,  36.23554993,  16.74915314,
        50.55737305, -58.35255432, -45.75421524,   6.07695007,
       -12.05615997, -53.71590805,  37.8598175 ,  20.60158539,
        49.38508606, -58.52809334,   5.41500092, -15.80286407,
       -54.89497757,  39.82655334,  20.60166931,  49.59532166,
       -58.92876434,   5.54166412, -16.02197266, -55.44865417,
        39.80172729,  19.97240448,  49.64668274, -58.79

In [33]:
present_lon_centroids = lon_centroids[present_events.values]
present_lon_centroids

array([202.16104126, 139.4487915 , 272.19537354, 328.63656616,
       213.7925415 , 174.93237305, 265.05838013, 324.3321228 ,
        73.9208374 ,          nan, 213.09558105, 177.53747559,
       264.85873413, 324.69134521, 109.09606171,  16.8902092 ,
       213.556427  , 175.69032288, 265.2144165 , 325.71078491,
       112.01115417,  17.2373848 , 194.19151306, 213.04167175,
       169.19593811, 266.97592163, 324.63909912, 112.2940979 ,
        18.14592552, 194.34596252, 215.84580994, 167.67546082,
       268.47003174, 324.4473877 , 115.43788147,  15.98383808,
       163.4641571 , 216.53622437, 165.85940552, 268.16183472,
       324.1355896 , 108.98645782,  17.76825905, 165.91818237,
       228.21195984, 220.94854736, 166.16447449, 274.03359985,
       322.0954895 , 116.02654266, 166.34750366, 222.39871216,
       215.42900085, 166.13002014, 274.47366333, 323.2928772 ,
       111.10618591, 170.00158691, 220.7149353 , 215.48898315,
       168.12075806, 275.05621338, 324.33236694, 110.94

In [31]:
(present_lat_centroids >= lat_min).all()

np.True_

In [32]:
(present_lat_centroids <= lat_max).all()

np.True_

In [ ]:
(present_lon_centroids >= lon_min).all()